In [ ]:
!pip install dotenv

In [2]:
# Importing Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint
import time
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
import numpy as np

In [3]:
os.getenv('password')

'4rFR!gPHpY4tU8y'

In [4]:
user = os.getenv('USER')

password = os.getenv('password')


In [5]:
#Create the browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

try:

    #Visit the website in 'url'
    url = 'https://www.appannie.com/dashboard/home'
    browser.visit(url)

    #Enter login info
    time.sleep(1)
    browser.find_by_name(name = 'username').type(user)
    browser.find_by_css('input')[1].type(password)
    browser.find_by_tag('button').click()

    #########
    def checker(func,go=None):
        #func is a 'is_present' boolean
        for i in range(3):
            if func:
                go
                break
            else:
                browser.reload()
                time.sleep(2)
    ##########

    #Check for presence of 'Top Chart' tab, click tab
    checker((browser.is_element_present_by_text('Top Charts',wait_time=2)),browser.find_by_text('Top Charts').click())

    #Create date range
    today = datetime.now()
    dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(5)]



    #https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
    #list of dictionaries, each one a different day
    list_of_builds = []

    for moment in dates:
        #get top chart info
        url = f'https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date={moment}&feed=All&page_number=0&page_size=500&rank_sorting_type=rank'
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')

        checker(browser.is_element_present_by_css('table'))

        all_trs = soup.table.tbody.find_all('tr')
        build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
        for _ in all_trs:
            current_rank_apps = [elem.text for elem in _.select('a span')]
            
            toss, app_id1, app_id2, app_id3 = [i.get('data-appid') for i in _.find_all('td')]
            
            rank_num = _.span.text
            build_df['Rank'].append((rank_num))
            build_df['Free'].append([current_rank_apps[0],current_rank_apps[1],app_id1])
            build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3],app_id2])
            build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5],app_id3])

        list_of_builds.append(build_df)

    #concatenate all dataframes
    builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
    sep3_dec1_t500_ios = pd.concat(builds_to_concat)
except Exception as e:
    browser.quit()
    print(e)
finally:
    browser.quit()

In [14]:
sep3_dec1_t500.to_csv('top_500_90day.csv')

In [6]:
sep3_dec1_t500_ios

,Rank,Free,Paid,Grossing
Date,,,,
2020-12-05,1,"[Hype Simulator, Ulkar Akhundzada, 1502153059]","[Minecraft, Mojang, 479516143]","[YouTube: Watch, Listen, Stream, Google, 54400..."
2020-12-05,2,"[Among Us!, InnerSloth, 1351168404]","[Heads Up! Best Charades game, Warner Bros, 62...","[Tinder - Dating & Make Friends, Match Group, ..."
2020-12-05,3,"[YouTube: Watch, Listen, Stream, Google, 54400...","[Plague Inc., Ndemic Creations, 525818839]","[Roblox, Roblox, 431946152]"
2020-12-05,4,"[TikTok, TikTok Pte Ltd, 835599320]","[Bloons TD 6, Ninja Kiwi, 1118115766]","[Disney+, Disney, 1446075923]"
2020-12-05,5,"[ZOOM Cloud Meetings, Zoom Video Communication...","[Procreate Pocket, Savage Interactive, 916366645]","[Pokémon GO, Niantic, 1094591345]"
...,...,...,...,...
2020-12-01,496,"[Steve - Widget Game, Ivan De Cabo, 1090617661]","[VeinSeek Pro, VeinSeek, 1174536386]","[Small Town Murders: Match 3, Rovio, 1479036032]"
2020-12-01,497,"[Earnin - Get $100, Activehours, 723815926]","[Due - Reminders & Timers, Phocus, 390017969]","[Traffic Puzzle, Huuuge Games, 1471999779]"
2020-12-01,498,"[Fonts Art - Fonts for iPhones, Ascella Mobile...","[System Status Pro: hw monitor, Techet, 401457...","[GoPro, GoPro, 561350520]"


In [7]:
df_raw_new = sep3_dec1_t500_ios.reset_index()
df_raw_new

,Date,Rank,Free,Paid,Grossing
0,2020-12-05,1,"[Hype Simulator, Ulkar Akhundzada, 1502153059]","[Minecraft, Mojang, 479516143]","[YouTube: Watch, Listen, Stream, Google, 54400..."
1,2020-12-05,2,"[Among Us!, InnerSloth, 1351168404]","[Heads Up! Best Charades game, Warner Bros, 62...","[Tinder - Dating & Make Friends, Match Group, ..."
2,2020-12-05,3,"[YouTube: Watch, Listen, Stream, Google, 54400...","[Plague Inc., Ndemic Creations, 525818839]","[Roblox, Roblox, 431946152]"
3,2020-12-05,4,"[TikTok, TikTok Pte Ltd, 835599320]","[Bloons TD 6, Ninja Kiwi, 1118115766]","[Disney+, Disney, 1446075923]"
4,2020-12-05,5,"[ZOOM Cloud Meetings, Zoom Video Communication...","[Procreate Pocket, Savage Interactive, 916366645]","[Pokémon GO, Niantic, 1094591345]"
...,...,...,...,...,...
2495,2020-12-01,496,"[Steve - Widget Game, Ivan De Cabo, 1090617661]","[VeinSeek Pro, VeinSeek, 1174536386]","[Small Town Murders: Match 3, Rovio, 1479036032]"
2496,2020-12-01,497,"[Earnin - Get $100, Activehours, 723815926]","[Due - Reminders & Timers, Phocus, 390017969]","[Traffic Puzzle, Huuuge Games, 1471999779]"
2497,2020-12-01,498,"[Fonts Art - Fonts for iPhones, Ascella Mobile...","[System Status Pro: hw monitor, Techet, 401457...","[GoPro, GoPro, 561350520]"
2498,2020-12-01,499,"[Flipgrid., Flipgrid, 756972930]","[MOODA, OLIVESTONE Lab., 1476256748]","[Coffee Meets Bagel Dating App, Coffee Meets B..."


In [8]:
def check_publisher(element, element_list):
    if element not in element_list:
        element_list.append(element)
        
def check_app(element, cat, element_list, cat_list, lookup_table, pub_id_list, app_id_list):
    if element[0] not in element_list:
        element_list.append(element[0])
        cat_list.append(cat)
        app_id_list.append(element[2])
        pub_id_list.append(int(lookup_table[lookup_table['Publisher_Name'] == element[1]]['Publisher_ID']))
        
    

In [12]:
publisher_list = []

for index, row in df_raw_new.iterrows():
    check_publisher(row['Free'][1], publisher_list)
    check_publisher(row['Grossing'][1], publisher_list)
    check_publisher(row['Paid'][1], publisher_list)
   



In [13]:
publisher_id = np.arange(1, len(publisher_list)+1)

In [14]:
publisher_list

['Ulkar Akhundzada',
 'Google',
 'Mojang',
 'InnerSloth',
 'Match Group',
 'Warner Bros',
 'Roblox',
 'Ndemic Creations',
 'TikTok Pte Ltd',
 'Disney',
 'Ninja Kiwi',
 'Zoom Video Communications',
 'Niantic',
 'Savage Interactive',
 'Instagram',
 'Pandora',
 'Marmalade',
 'Facebook',
 'King',
 'RobTop',
 'BANDAI NAMCO Entertainment',
 'Rockstar Games',
 'Snap',
 'Warner Media',
 'Lightricks',
 'Voodoo',
 'Hulu',
 'HotSchedules',
 'Bumble',
 'True Axis',
 'SayGames',
 'Twitch Interactive',
 'Suture Sound Inc.',
 'Square Inc',
 'Playrix',
 'BoltCreative',
 'Amazon',
 'Flipline',
 'Activision Publishing',
 'ADVA Soft',
 'Spotify',
 'Supercell',
 'Clickteam',
 'Netflix',
 '505 Games',
 'WhatsApp',
 'BIGO',
 'So Far So Good',
 'Shopify',
 'Moon Active',
 'Seekrtech',
 'miHoYo',
 'Terminal Eleven',
 'Peak',
 'Domus Technica',
 'Garena Online',
 'DoorDash',
 'Tencent',
 '2kit Consulting',
 'Discord',
 'Giants Software',
 'Zynga',
 'Playtika',
 '2K Games',
 'PayPal',
 'Bubblegum Games',
 'Chuc

In [15]:
df_publisher = pd.DataFrame(zip(publisher_id, publisher_list), columns=['Publisher_ID','Publisher_Name'])
df_publisher

,Publisher_ID,Publisher_Name
0,1,Ulkar Akhundzada
1,2,Google
2,3,Mojang
3,4,InnerSloth
4,5,Match Group
...,...,...
1218,1219,Amp Me
1219,1220,AM APPS
1220,1221,Lumos Labs
1221,1222,Phocus


In [16]:
app_list = []
cat_list = []
pub_id = []
app_id = []

for index, row in df_raw_new.iterrows():
    # If the app is not in the list yet, insert it into the list and insert the category on the cat_list: 1(Free), 2(Grossing), 3(Paid)
    check_app(row['Free'], 1, app_list, cat_list, df_publisher, pub_id, app_id)
    check_app(row['Grossing'], 2, app_list, cat_list, df_publisher, pub_id, app_id)
    check_app(row['Paid'], 3, app_list, cat_list, df_publisher, pub_id, app_id)

In [17]:
df_app = pd.DataFrame(zip(app_id, app_list, cat_list, pub_id), columns=['App_ID','App_Name','Category','Publisher_ID'])
df_app

,App_ID,App_Name,Category,Publisher_ID
0,1502153059,Hype Simulator,1,1
1,544007664,"YouTube: Watch, Listen, Stream",2,2
2,479516143,Minecraft,3,3
3,1351168404,Among Us!,1,4
4,547702041,Tinder - Dating & Make Friends,2,5
...,...,...,...,...
1745,1471999779,Traffic Puzzle,2,162
1746,390017969,Due - Reminders & Timers,3,1222
1747,1480383173,Fonts Art - Fonts for iPhones,1,722
1748,401457165,System Status Pro: hw monitor,3,768


In [18]:
df_free = pd.DataFrame(df_raw_new['Free'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_free

,App_Name,Publisher,App_ID
0,Hype Simulator,Ulkar Akhundzada,1502153059
1,Among Us!,InnerSloth,1351168404
2,"YouTube: Watch, Listen, Stream",Google,544007664
3,TikTok,TikTok Pte Ltd,835599320
4,ZOOM Cloud Meetings,Zoom Video Communications,546505307
...,...,...,...
2495,Steve - Widget Game,Ivan De Cabo,1090617661
2496,Earnin - Get $100,Activehours,723815926
2497,Fonts Art - Fonts for iPhones,Ascella Mobile,1480383173
2498,Flipgrid.,Flipgrid,756972930


In [19]:
df_free_cleaned = df_raw_new[['Date','Rank']].merge(df_free[['App_ID']], left_index = True, right_index = True)
df_free_cleaned

,Date,Rank,App_ID
0,2020-12-05,1,1502153059
1,2020-12-05,2,1351168404
2,2020-12-05,3,544007664
3,2020-12-05,4,835599320
4,2020-12-05,5,546505307
...,...,...,...
2495,2020-12-01,496,1090617661
2496,2020-12-01,497,723815926
2497,2020-12-01,498,1480383173
2498,2020-12-01,499,756972930


In [20]:
df_paid = pd.DataFrame(df_raw_new['Paid'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_paid

,App_Name,Publisher,App_ID
0,Minecraft,Mojang,479516143
1,Heads Up! Best Charades game,Warner Bros,623592465
2,Plague Inc.,Ndemic Creations,525818839
3,Bloons TD 6,Ninja Kiwi,1118115766
4,Procreate Pocket,Savage Interactive,916366645
...,...,...,...
2495,VeinSeek Pro,VeinSeek,1174536386
2496,Due - Reminders & Timers,Phocus,390017969
2497,System Status Pro: hw monitor,Techet,401457165
2498,MOODA,OLIVESTONE Lab.,1476256748


In [21]:
df_paid_cleaned = df_raw_new[['Date','Rank']].merge(df_paid[['App_ID']], left_index = True, right_index = True)
df_paid_cleaned

,Date,Rank,App_ID
0,2020-12-05,1,479516143
1,2020-12-05,2,623592465
2,2020-12-05,3,525818839
3,2020-12-05,4,1118115766
4,2020-12-05,5,916366645
...,...,...,...
2495,2020-12-01,496,1174536386
2496,2020-12-01,497,390017969
2497,2020-12-01,498,401457165
2498,2020-12-01,499,1476256748


In [22]:
df_grossing = pd.DataFrame(df_raw_new['Grossing'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_grossing

,App_Name,Publisher,App_ID
0,"YouTube: Watch, Listen, Stream",Google,544007664
1,Tinder - Dating & Make Friends,Match Group,547702041
2,Roblox,Roblox,431946152
3,Disney+,Disney,1446075923
4,Pokémon GO,Niantic,1094591345
...,...,...,...
2495,Small Town Murders: Match 3,Rovio,1479036032
2496,Traffic Puzzle,Huuuge Games,1471999779
2497,GoPro,GoPro,561350520
2498,Coffee Meets Bagel Dating App,Coffee Meets Bagel,630119301


In [23]:
df_grossing_cleaned = df_raw_new[['Date','Rank']].merge(df_grossing[['App_ID']], left_index = True, right_index = True)
df_grossing_cleaned

,Date,Rank,App_ID
0,2020-12-05,1,544007664
1,2020-12-05,2,547702041
2,2020-12-05,3,431946152
3,2020-12-05,4,1446075923
4,2020-12-05,5,1094591345
...,...,...,...
2495,2020-12-01,496,1479036032
2496,2020-12-01,497,1471999779
2497,2020-12-01,498,561350520
2498,2020-12-01,499,630119301


In [5]:
#Create the browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

try:

    #Visit the website in 'url'
    url = 'https://www.appannie.com/dashboard/home'
    browser.visit(url)

    #Enter login info
    time.sleep(1)
    browser.find_by_name(name = 'username').type(os.getenv('user'))
    browser.find_by_css('input')[1].type(os.getenv('password'))
    browser.find_by_tag('button').click()

    #########
    def checker(func,go=None):
        #func is a 'is_present' boolean
        for i in range(3):
            if func:
                go
                break
            else:
                browser.reload()
                time.sleep(2)
    ##########

    #Check for presence of 'Top Chart' tab, click tab
    checker((browser.is_element_present_by_text('Top Charts',wait_time=2)),browser.find_by_text('Top Charts').click())

    #Create date range
    today = datetime.now()
    dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(90)]



    #https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
    #list of dictionaries, each one a different day
    list_of_builds = []

    for moment in dates:
        #get top chart info
        url = f'https://www.appannie.com/apps/google-play/top-chart/?country=US&category=1&device=&date={moment}&feed=All&rank_sorting_type=rank&page_number=0&page_size=100'
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')

        checker(browser.is_element_present_by_css('table'))

        all_trs = soup.table.tbody.find_all('tr')
        build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
        for _ in all_trs:
            current_rank_apps = [elem.text for elem in _.select('a span')]
            
            toss, app_id1, app_id2, app_id3, app_id4, app_id5 = [i.get('data-appid') for i in _.find_all('td')]
            
            rank_num = _.span.text
            build_df['Rank'].append((rank_num))
            build_df['Free'].append([current_rank_apps[0],current_rank_apps[1],app_id1])
            build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3],app_id2])
            build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5],app_id3])

        list_of_builds.append(build_df)

    #concatenate all dataframes
    builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
    sep3_dec1_t500_google_play = pd.concat(builds_to_concat)
except Exception as e:
    browser.quit()
    print(e)
finally:
    browser.quit()

In [1]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://@localhost:5432/mydatabase')

In [2]:
engine

Engine(postgresql://@localhost:5432/mydatabase)